In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,765 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.ne

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-21 05:17:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.68MB/s    in 0.2s    

2022-05-21 05:17:24 (5.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vine_review_analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13780696|             1|
|   48489393|             8|
|   43380682|             1|
|   17133428|             1|
|    9731896|             2|
|    1797768|             1|
|   27446622|             1|
|     120706|             3|
|   15915947|             2|
|   37520920|             1|
|   43920023|             9|
|   14309190|             8|
|   22316505|             1|
|   43949054|             2|
|   21102747|             1|
|   19021463|             1|
|   16075459|             8|
|   51475782|             2|
|   11555870|             5|
|   35471383|             3|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title']).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01427DDXM|Zoozie LA Women's...|
|B013TPITFA|Tabeez Women's Pl...|
|B013FU4Y9E|Ekouaer Women's U...|
|B013FNK6PC|Ezcosplay Women O...|
|B013FACXVK|Womens Two Piece ...|
|B013DHKQ3W|OURS Women's 3 4 ...|
|B0131D4NKU|Genie Women's Jeg...|
|B0131CA9K4|BeWild Brand - Ju...|
|B0130ZDFTE|Flora&Fred Women'...|
|B012Y665WY|AW Fashion's I Ne...|
|B012X0G1KW|KAMACO Women's Se...|
|B012VOPU84|iLoveSIA Women's ...|
|B012UUEWZG|Harley-Davidson W...|
|B012LQU4CE|Women's Laced & L...|
|B012ERJSKO|Amoin Style Women...|
|B012ADLW92|Cocoship Retro 50...|
|B012980QUY|Sanwood® Cute Bab...|
|B0125V8SZK|OURS Women's Comf...|
|B0125M6TZ0|Modern Kiwi Solid...|
|B01254G39K|Alberto Cardinali...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent',to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154| 2013-01-14|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556| 2014-03-04|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549| 2015-07-12|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728| 2015-06-03|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422| 2015-06-12|
|R1C4QH63NFL5NJ|   26631939|B01FWRXN0Y|     366144407| 2014-04-22|
|R2GP65O1U9N7BP|   48785098|B01EXNH1HE|     786052021| 2015-07-28|
|R3O29CT5MQQ3XQ|   39548589|B01E7OL09O|     108920964| 2015-07-10|
|R1ZECD2AA8QFF6|   29355866|B01DXHX81O|     317132458| 2015-08-09|
|R2S79GCF6J89OA|   27477484|B01DDULIJK|     110598191| 2014-05-24|
|R1CBCUGNP37MPK|   17685865|B01BOKOL4A|     121408017| 2015-06-10|
|R3NU4WUR5ZNV1V|   19787539|B01B3Q4Q0O|     134506082| 2013-09

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

Deliverable 2

In [19]:
filtered_df = vine_df.filter('total_votes>=20')
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|
|R2P76PJFUGXBHO|          1|            6|         45|   N|                N|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|
|R3M02FSD3BLUPU|          5|           19|         21|   N|                N|
|R1P0LEEJHH09L3|          5|           32|         32|   N|                Y|
|R1V6UU1EHW3Q12|          5|           57|         62|   N|     

In [20]:
filter1_df = filtered_df.withColumn('helpful_votes/total_votes',filtered_df['helpful_votes']/filtered_df['total_votes'])
filter2_df = filter1_df.filter('helpful_votes/total_votes>=0.5')
filter2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes/total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|                   0.9375|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|       0.9807692307692307|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|       0.8055555555555556|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|       0.9866666666666667|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|       0.8846153846153846|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|       0.9251700680272109|
|R3M02FSD3BLUPU|          5|           19|         21|   N|     

In [21]:
filterY_df = filter2_df.filter(filter2_df["vine"] == "Y")
filterY_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes/total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
| R6U9701C3BGO6|          3|          139|        147|   Y|                N|       0.9455782312925171|
|R1XK3ALB45D7N4|          5|           33|         34|   Y|                N|       0.9705882352941176|
|R1IZCSTLX81D6C|          5|           31|         33|   Y|                N|       0.9393939393939394|
|R2C8NC8EQLH4JF|          3|           45|         48|   Y|                N|                   0.9375|
|R1JJ1YOJMOML1P|          5|           18|         21|   Y|                N|       0.8571428571428571|
|R3TKG664L9MTXJ|          4|          164|        175|   Y|                N|       0.9371428571428572|
|R2E942L5EX73FP|          5|           29|         33|   Y|     

In [22]:
filterY_df.count()

33

In [23]:
filterN_df = filter2_df.filter(filter2_df["vine"] == "N")
filterN_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes/total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
|R35PT06NWP7LDP|          5|           30|         32|   N|                N|                   0.9375|
| RQW4AFOG9MR4Z|          5|           51|         52|   N|                N|       0.9807692307692307|
|R2SMUEBMGLAJQK|          5|           29|         36|   N|                Y|       0.8055555555555556|
|R1XVIZZALU5P6J|          5|          148|        150|   N|                Y|       0.9866666666666667|
|R11UFMG8M2488I|          4|           23|         26|   N|                N|       0.8846153846153846|
|R2OSHKSPXU68W5|          5|          136|        147|   N|                N|       0.9251700680272109|
|R3M02FSD3BLUPU|          5|           19|         21|   N|     

In [24]:
filterN_df.count()

45388

In [25]:
paid1_df = filterY_df.groupby('star_rating').agg({"star_rating":'count'})
paid1_df.orderBy(paid1_df['star_rating'].desc()).show()

+-----------+------------------+
|star_rating|count(star_rating)|
+-----------+------------------+
|          5|                15|
|          4|                 7|
|          3|                10|
|          2|                 1|
+-----------+------------------+



In [26]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
paid_df = paid1_df.withColumn('total', f.sum('count(star_rating)').over(Window.partitionBy()))\
  .withColumn('percentage', f.col('count(star_rating)')/f.col('total'))
paid_df.orderBy(paid_df['star_rating'].desc()).show()

+-----------+------------------+-----+--------------------+
|star_rating|count(star_rating)|total|          percentage|
+-----------+------------------+-----+--------------------+
|          5|                15|   33| 0.45454545454545453|
|          4|                 7|   33| 0.21212121212121213|
|          3|                10|   33| 0.30303030303030304|
|          2|                 1|   33|0.030303030303030304|
+-----------+------------------+-----+--------------------+



In [27]:
unpaid1_df = filterN_df.groupby('star_rating').agg({"star_rating":'count'})
unpaid_df = unpaid1_df.withColumn('total', f.sum('count(star_rating)').over(Window.partitionBy()))\
  .withColumn('percentage', f.col('count(star_rating)')/f.col('total'))
unpaid_df.orderBy(unpaid_df['star_rating'].desc()).show()

+-----------+------------------+-----+-------------------+
|star_rating|count(star_rating)|total|         percentage|
+-----------+------------------+-----+-------------------+
|          5|             23733|45388| 0.5228915131752886|
|          4|              7803|45388|0.17191768749449193|
|          3|              4388|45388|0.09667753591257601|
|          2|              3085|45388|0.06796950735877325|
|          1|              6379|45388|0.14054375605887018|
+-----------+------------------+-----+-------------------+

